In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('E:/7 th sem/projects/Chatbot-using-Python-master/intents.json').read()
intents = json.loads(data_file)

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [4]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

57 documents
13 classes ['activities', 'admission', 'course', 'distance_learning', 'eligibility', 'exam', 'fees', 'goodbye', 'greeting', 'hostel', 'placement', 'scholarships', 'thanks']
91 unique lemmatized words ["'s", ',', 'a', 'activity', 'admission', 'an', 'and', 'anyone', 'apply', 'applying', 'are', 'at', 'available', 'awesome', 'b.tech', 'bye', 'chatting', 'club', 'conduct', 'course', 'criterion', 'day', 'degree', 'distance', 'do', 'doe', 'education', 'eligibility', 'entrance', 'event', 'exam', 'extracurricular', 'facility', 'fee', 'for', 'get', 'good', 'goodbye', 'have', 'hello', 'helpful', 'helping', 'hey', 'hi', 'hola', 'hostel', 'how', 'i', 'in', 'is', 'jain', 'job', 'later', 'learning', 'like', 'living', 'master', 'me', 'much', 'next', 'nice', 'of', 'offer', 'online', 'opportunity', 'placement', 'popular', 'postgraduate', 'process', 'program', 'record', 'requirement', 'scholarship', 'scope', 'see', 'step', 'structure', 'student', 'test', 'thank', 'thanks', 'that', 'the', 'th

In [5]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

     # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [6]:
import random
import numpy as np

# Sample check for training list
for i, element in enumerate(training):
    if len(element) != 2:
        print(f"Element at index {i} is not of length 2. Length found: {len(element)}")
        # Optional: print the problematic element for debugging
        print("Problematic element:", element)

# Ensure each element has two consistent sub-elements (e.g., X and Y).
random.shuffle(training)

# Turn training into a NumPy array of type 'object' if elements are of varying sizes or types.
training = np.array(training, dtype=object)

# Create train and test lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])

print("Training data created")


Training data created


In [7]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

C:\Users\User\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# Compile model with SGD optimizer
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fit and save the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("Model created")

C:\Users\User\anaconda3\Lib\site-packages\keras\src\optimizers\base_optimizer.py:34: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Epoch 1/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.0338 - loss: 2.6741   
Epoch 2/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0742 - loss: 2.5237 
Epoch 3/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2264 - loss: 2.4221 
Epoch 4/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1311 - loss: 2.4512     
Epoch 5/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3822 - loss: 2.2901 
Epoch 6/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3082 - loss: 2.2900 
Epoch 7/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3321 - loss: 2.1198 
Epoch 8/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3272 - loss: 2.0295 
Epoch 9/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4735 - loss: 1.9412 
Epoch 10/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4458 - loss: 1.8979 
Epoch 11/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5538 - loss: 1.6952 
Epoch 12/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3m

Model created
